In [41]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import joblib

In [42]:
data=pd.read_csv("csv file1.csv",error_bad_lines=False, encoding="latin-1")
data.head()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_5148\3476389009.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data=pd.read_csv("csv file1.csv",error_bad_lines=False, encoding="latin-1")
b'Skipping line 45748: expected 10 fields, saw 11\n'
b'Skipping line 134013: expected 10 fields, saw 14\n'
b'Skipping line 267355: expected 10 fields, saw 15\n'
b'Skipping line 353041: expected 10 fields, saw 12\n'
b'Skipping line 441018: expected 10 fields, saw 11\n'
b'Skipping line 485619: expected 10 fields, saw 13\n'
b'Skipping line 662577: expected 10 fields, saw 16\n'
b'Skipping line 753251: expected 10 fields, saw 11\n'
b'Skipping line 840719: expected 10 fields, saw 15\n'
b'Skipping line 1065529: expected 10 fields, saw 14\n'
b'Skipping line 1242887: expected 10 fields, saw 11\n'
b'Skipping line 1329814: expected 10 fields, saw 14\nSkipping line 1374246: expected 10 fields, saw 11\n'
b'Skipping line 

,City,Place,Review,Rating,Name,Date,Raw_Review,City.1,userId,Unnamed: 9
0,Aamby Valley City,19 Degree North,aamby valley beautiful place clear blue skies ...,5,Anonymous,NaN,Aamby valley is a beautiful place with its cle...,Aamby Valley City,1.000,NaN
1,Aamby Valley City,19 Degree North,executed obt akshay thanx team thoroughly enjo...,4,Anonymous,NaN,Very well executed obt by Akshay.... Thanx as ...,Aamby Valley City,2.000,NaN
2,Aamby Valley City,19 Degree North,awesome experience atv tracts obstacles mainta...,5,Anonymous,NaN,Awesome experience at the ATV\nTracts and obst...,Aamby Valley City,2.000,NaN
3,Aamby Valley City,19 Degree North,visited aamby valley yesterday short excursion...,4,Anonymous,NaN,we visited the Aamby Valley yesterday for shor...,Aamby Valley City,3.000,NaN
4,Aamby Valley City,19 Degree North,far mumbai place finest adventure places visit...,5,Anonymous,NaN,"Not far from Mumbai, this place is one of the ...",Aamby Valley City,4.000,NaN


In [43]:
data=data.drop(["Review","Name","Date","Raw_Review","City.1","City","Unnamed: 9"],axis=1)
data

In [ ]:
data.isna().sum()

In [44]:
data=data.dropna()


In [45]:
data = data[~data.duplicated(subset=['userId', 'Place'], keep=False)]

data.to_csv('cleaned_dataset.csv', index=False)


In [46]:
data=pd.read_csv('cleaned_dataset.csv')
data.shape

(23224, 3)

In [47]:
data["userId"].isna().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of 0        False
1        False
2        False
3        False
4        False
         ...  
23219    False
23220    False
23221    False
23222    False
23223    False
Name: userId, Length: 23224, dtype: bool>

In [48]:
data.isna().sum()


Place     0
Rating    0
userId    0
dtype: int64

In [49]:
data.shape

(23224, 3)

In [99]:
Place_sum=list(data.Place.unique())
len(Place_sum)

154

In [53]:
combine_Place_rating = data.dropna(axis=0, subset=['Place'])

Place_ratingCount = combine_City_rating.groupby(by=['Place'])['Rating'].count().reset_index().rename(columns={'Rating': 'totalRatingCount'})[['Place', 'totalRatingCount']]

Place_ratingCount.head()

,Place,totalRatingCount
0,19 Degree North,15
1,AUGUSTIN Studio & Gallery,5
2,Achirupakkam,1
3,Adalaj Step-well,1
4,Adichikkavu Sree Durga Devi Kshetram Temple,5


In [54]:
combine_Place_rating

,Place,Rating,userId
0,19 Degree North,5,1.000
1,19 Degree North,4,3.000
2,19 Degree North,5,4.000
3,19 Degree North,5,5.000
4,19 Degree North,5,6.000
...,...,...,...
23219,Taj Mahal,5,18594.000
23220,Taj Mahal,4,18595.000
23221,Taj Mahal,5,18596.000
23222,Adalaj Step-well,4,1662.000


In [56]:
rating_with_totalRatingCount = combine_Place_rating.merge(Place_ratingCount, left_on='Place', right_on='Place', how='left')
rating_with_totalRatingCount.userId.unique().sum()

172914906.0

In [57]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(Place_ratingCount['totalRatingCount'].describe())

count    202.000
mean     138.371
std      758.223
min        1.000
25%        4.000
50%        9.500
75%       41.500
max     8570.000
Name: totalRatingCount, dtype: float64


In [58]:
popularity_threshold = 10
rating_popular_Place = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_Place.head()

,Place,Rating,userId,totalRatingCount
0,19 Degree North,5,1.000,15
1,19 Degree North,4,3.000,15
2,19 Degree North,5,4.000,15
3,19 Degree North,5,5.000,15
4,19 Degree North,5,6.000,15


In [59]:
rating_popular_Place.shape

(22888, 4)

In [60]:
data["userId"][1]

3.0

In [61]:
Place_features_df = rating_popular_Place.pivot_table(index='Place', columns='userId', values='Rating').fillna(0)
Place_features_df.head()

userId,1.000,2.000,3.000,4.000,5.000,6.000,7.000,8.000,9.000,10.000,...,18587.000,18588.000,18589.000,18590.000,18591.000,18592.000,18593.000,18594.000,18595.000,18596.000
Place,,,,,,,,,,,,,,,,,,,,,
19 Degree North,5.000,0.000,4.000,5.000,5.000,5.000,4.000,3.000,5.000,5.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Agonda Beach,4.000,4.000,5.000,5.000,5.000,5.000,5.000,3.000,5.000,4.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Agra Art Gallery,5.000,2.000,5.000,5.000,5.000,5.000,5.000,4.000,4.000,3.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Agra Fort,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
Aloha Surf India,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [63]:
Place_features_df.shape

(75, 18486)

In [65]:
from scipy.sparse import csr_matrix

Place_features_df_matrix = csr_matrix(Place_features_df.values)


In [67]:
print(Place_features_df)
print(Place_features_df.shape)


userId                       1.000      2.000      3.000      4.000      \
Place                                                                     
19 Degree North                  5.000      0.000      4.000      5.000   
Agonda Beach                     4.000      4.000      5.000      5.000   
Agra Art Gallery                 5.000      2.000      5.000      5.000   
Agra Fort                        0.000      0.000      0.000      0.000   
Aloha Surf India                 0.000      0.000      0.000      0.000   
...                                ...        ...        ...        ...   
Tripura Government Museum        0.000      0.000      0.000      0.000   
Ujjayanta Palace                 0.000      0.000      0.000      0.000   
Unakoti                          0.000      0.000      0.000      0.000   
Venuban Vihar Buddha Shrine      0.000      0.000      0.000      0.000   
Wayanad Tea Museum               4.000      4.000      5.000      3.000   

userId                  

In [69]:
Place_features_df.isna().sum()


userId
1.000        0
2.000        0
3.000        0
4.000        0
5.000        0
            ..
18592.000    0
18593.000    0
18594.000    0
18595.000    0
18596.000    0
Length: 18486, dtype: int64

In [71]:

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(Place_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [83]:

available_Place = list(Place_features_df.index)
print("Available Place:")

for i, Place in enumerate(available_Place):
    print("{0}: {1}".format(i+1, Place))


selected_Place_index = int(input("\nSelect a Place (enter a number from 1 to {0}): ".format(len(available_Place))))


selected_Place_name = available_Place[selected_Place_index - 1]


print("\nYou selected: {0}\n".format(selected_Place_name))


query_index = Place_features_df.index.get_loc(selected_Place_name)
distances, indices = model_knn.kneighbors(Place_features_df.iloc[query_index, :].values.reshape(1, -1), n_neighbors=6)


for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(selected_Place_name))
    else:
        print('{0}: {1}'.format(i, Place_features_df.index[indices.flatten()[i]]))


Available Place:
1: 19 Degree North
2: Agonda Beach
3: Agra Art Gallery
4: Agra Fort
5: Aloha Surf India
6: Anguri Bagh
7: Animal Shelter Agonda
8: Basar Saraswati Temple
9: Bio Veda Ayurvedic Beach Resort Spa
10: Chand Baori (Step well)
11: Chapoli Dam
12: Chini Ka Rauza
13: Chini Ka Rauza Mausoleum
14: Dargah Shariff
15: Dayal Bagh
16: Diwan-i-Am Agra Fort
17: Dolphin The Water World
18: Fluidsymmetry
19: Fourteen Goddess Temple
20: Galgibaga Beach
21: Gurudwara Guru ka Tal
22: Harshat Mata Temple
23: Heritage Park
24: Indian Spices & Herbal Garden
25: Island of Thinnakara
26: Jagannath Temple
27: Jahangir Mahal
28: Jama Masjid
29: Jampui Hill
30: Jawab Masjid
31: Kadam Dam
32: Kalakriti Cultural & Convention Center
33: Kalyan Sagar Lake
34: Kamaleswari Temple
35: Kawal Wildlife Sanctuary
36: Keetham Lake
37: Korai Village Agra
38: Krishna Ayurveda Massage Centre
39: Kuntala Waterfall
40: Lakshadweep Tour Package - Day Tours
41: Mariam's Tomb
42: Mehtab Bagh
43: Mother Teresa's Missi